In [7]:
# Social Network Ads: https://www.kaggle.com/datasets/rakeshrau/social-network-ads/data

# Loading the data
import pandas as pd
import networkx as nx

url = "https://raw.githubusercontent.com/hbedros/centrality-measures/main/data/Social_Network_Ads.csv"
data = pd.read_csv(url)

# Display the first few rows of the dataset
print(data.head())

# Get basic information about the dataset
print(data.info())

# Check for any missing values
print(data.isnull().sum())

# Summary statistics of the dataset
print(data.describe())

    User ID  Gender  Age  EstimatedSalary  Purchased
0  15624510    Male   19            19000          0
1  15810944    Male   35            20000          0
2  15668575  Female   26            43000          0
3  15603246  Female   27            57000          0
4  15804002    Male   19            76000          0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   User ID          400 non-null    int64 
 1   Gender           400 non-null    object
 2   Age              400 non-null    int64 
 3   EstimatedSalary  400 non-null    int64 
 4   Purchased        400 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 15.8+ KB
None
User ID            0
Gender             0
Age                0
EstimatedSalary    0
Purchased          0
dtype: int64
            User ID         Age  EstimatedSalary   Purchased
count  4.000000e+02  400.000

In [8]:
# Create a graph from the data
g = nx.Graph()

# Add nodes with attributes
for index, row in data.iterrows():
    g.add_node(row['User ID'], gender=row['Gender'], age=row['Age'], salary=row['EstimatedSalary'], purchased=row['Purchased'])

# Hypothetical edges: Connect users if they have the same age and similar salary
for i, node1 in enumerate(g.nodes(data=True)):
    for j, node2 in enumerate(g.nodes(data=True)):
        if i != j and node1[1]['age'] == node2[1]['age'] and abs(node1[1]['salary'] - node2[1]['salary']) <= 10000:
            g.add_edge(node1[0], node2[0])

# Calculate degree centrality
degree_centrality = nx.degree_centrality(g)

# Calculate eigenvector centrality
eigenvector_centrality = nx.eigenvector_centrality(g)

# Add centrality measures to the dataset
data['degree_centrality'] = data['User ID'].map(degree_centrality)
data['eigenvector_centrality'] = data['User ID'].map(eigenvector_centrality)

# Display the updated dataset with centrality measures
print(data.head())

PowerIterationFailedConvergence: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 100 iterations')